In [45]:
%run "00 - Automatic evaluation - DEV.py"

In [46]:
df_all = pd.merge(scores_all, params_df, left_index=True, right_index=True)

In [53]:
df_all[['bleu', 'meteor', 'ter']].describe()

,bleu,meteor,ter
count,243.000000,243.000000,243.000000
mean,60.004856,0.436822,0.371107
std,0.780190,0.005001,0.003229
min,58.450000,0.426882,0.362833
25%,59.270000,0.431635,0.368537
50%,60.170000,0.438581,0.371218
75%,60.660000,0.440935,0.373461
max,61.410000,0.445059,0.378288


In [47]:
df_all[['dp_scorer_n', 'sa_scorer_n', 'tems_lm_n', 'referrer_lm_n', 'txs_lm_n', 'bleu', 'meteor', 'ter']].sort_values('bleu', ascending=False).head()

,dp_scorer_n,sa_scorer_n,tems_lm_n,referrer_lm_n,txs_lm_n,bleu,meteor,ter
-4715630725955452178,4,4,5,3,5,61.41,0.445059,0.366977
-5618203234013463585,4,4,4,3,5,61.30,0.443609,0.367562
4793302171907916260,3,3,5,3,5,61.24,0.444567,0.367659
4531560406721554503,4,4,5,4,5,61.22,0.444712,0.367318
1081957276066798553,3,3,4,3,5,61.18,0.443310,0.367611


In [54]:
df_all[['dp_scorer_n', 'sa_scorer_n', 'tems_lm_n', 'referrer_lm_n', 'txs_lm_n', 'bleu', 'meteor', 'ter']].sort_values('meteor', ascending=False).head()

,dp_scorer_n,sa_scorer_n,tems_lm_n,referrer_lm_n,txs_lm_n,bleu,meteor,ter
-4715630725955452178,4,4,5,3,5,61.41,0.445059,0.366977
4531560406721554503,4,4,5,4,5,61.22,0.444712,0.367318
4793302171907916260,3,3,5,3,5,61.24,0.444567,0.367659
4676793047550571015,4,3,5,3,5,61.18,0.444027,0.368439
-5775219471722982611,3,4,5,3,5,60.91,0.444003,0.370292


In [55]:
df_all[['dp_scorer_n', 'sa_scorer_n', 'tems_lm_n', 'referrer_lm_n', 'txs_lm_n', 'bleu', 'meteor', 'ter']].sort_values('ter').head()

,dp_scorer_n,sa_scorer_n,tems_lm_n,referrer_lm_n,txs_lm_n,bleu,meteor,ter
-3751447153309992170,4,5,5,3,4,60.59,0.441487,0.362833
2146993640707683977,4,4,5,3,4,60.87,0.442193,0.363466
-1482274546210820297,4,5,5,4,4,60.42,0.440621,0.363710
1100448053830229035,4,5,4,3,4,60.73,0.440678,0.364052
1245602715134337520,4,4,4,3,4,61.07,0.441544,0.364100


In [48]:
df_all.groupby('dp_scorer_n').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
dp_scorer_n,,,,,,,,
3,81.0,60.046667,0.689661,58.72,59.55,60.17,60.66,61.24
4,81.0,60.069383,0.834210,58.49,59.25,60.24,60.73,61.41
5,81.0,59.898519,0.807673,58.45,59.10,60.05,60.59,61.17


In [49]:
df_all.groupby('sa_scorer_n').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
sa_scorer_n,,,,,,,,
3,81.0,59.957407,0.770842,58.45,59.29,60.06,60.61,61.24
4,81.0,60.085802,0.799997,58.56,59.34,60.30,60.73,61.41
5,81.0,59.971358,0.772701,58.51,59.23,60.20,60.60,61.12


In [50]:
df_all.groupby('tems_lm_n').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
tems_lm_n,,,,,,,,
3,81.0,59.825679,0.788782,58.45,59.08,59.93,60.53,61.15
4,81.0,60.104444,0.701614,58.57,59.60,60.22,60.69,61.30
5,81.0,60.084444,0.823351,58.59,59.25,60.37,60.69,61.41


In [51]:
df_all.groupby('referrer_lm_n').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
referrer_lm_n,,,,,,,,
3,81.0,60.314691,0.695678,59.02,59.68,60.53,60.86,61.41
4,81.0,59.962593,0.798063,58.46,59.13,60.17,60.67,61.22
5,81.0,59.737284,0.740905,58.45,58.90,59.92,60.31,60.89


In [52]:
df_all.groupby('txs_lm_n').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
txs_lm_n,,,,,,,,
3,81.0,59.066543,0.377578,58.45,58.78,59.03,59.25,60.12
4,81.0,60.193951,0.345350,59.41,59.92,60.20,60.43,61.07
5,81.0,60.754074,0.288950,60.05,60.60,60.73,60.91,61.41
